# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import pickle

In [2]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [4]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fecec888e48>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fecec8d5978>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, num_epochs=0)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |



NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fecec888e48>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fecec8d5978>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.44640000000000002

In [11]:
net.fit(X, y, num_epochs=20)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       1 |      0.21232 | 0.04566 |
|       2 |      0.14647 | 0.03929 |
|       3 |      0.14004 | 0.03765 |
|       4 |      0.13640 | 0.03826 |
|       5 |      0.13370 | 0.03846 |
|       6 |      0.13143 | 0.03796 |
|       7 |      0.12951 | 0.03639 |
|       8 |      0.12779 | 0.03718 |
|       9 |      0.12617 | 0.03814 |
|      10 |      0.12471 | 0.03813 |
|      11 |      0.12331 | 0.03738 |
|      12 |      0.12198 | 0.03968 |
|      13 |      0.12063 | 0.03666 |
|      14 |      0.11929 | 0.03854 |
|      15 |      0.11806 | 0.03745 |
|      16 |      0.11684 | 0.03666 |
|      17 |      0.11562 | 0.03806 |
|      18 |      0.11445 | 0.03767 |
|      19 |      0.11333 | 0.04134 |
|      20 |      0.11

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fecec888e48>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fecec8d5978>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [12]:
y_proba = net.predict_proba(X)

In [13]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.9718

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [14]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [15]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [16]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [17]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [18]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.943611 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.957409 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.947179 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.953209 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.968206 -   0.5s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hi

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   19.2s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=Fa...ject at 0x7fecec8d5978>,),
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_epochs': [5, 10], 'update__learning_rate': [0.1, 0.5], 'hidden__num_units': [50, 100]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [19]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 100, 'max_epochs': 5, 'update__learning_rate': 0.5},
 0.95799999999999996)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [20]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [21]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [22]:
net = NeuralNetRegressor(l2, verbose=1)

In [23]:
net.set_params(update__learning_rate=0.0001)

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fecec89a128>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fecec89a0f0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

### Train

In [24]:
net.fit(X, y, num_epochs=20)

# Neural Network with 20401 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |    100 |
|   1 | dense  |    200 |
|   2 | dense  |      1 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       1 |  27564.93750 | 0.06610 |
|       2 |  26635.29102 | 0.03152 |
|       3 |  23243.69141 | 0.03158 |
|       4 |  13730.35645 | 0.03147 |
|       5 |   3490.96948 | 0.03153 |
|       6 |    578.24799 | 0.03169 |
|       7 |    262.88669 | 0.03325 |
|       8 |    218.15894 | 0.03102 |
|       9 |    200.76053 | 0.02980 |
|      10 |    188.74666 | 0.03279 |
|      11 |    178.77943 | 0.03013 |
|      12 |    170.02246 | 0.03224 |
|      13 |    162.08340 | 0.03268 |
|      14 |    154.73958 | 0.03173 |
|      15 |    147.86694 | 0.03092 |
|      16 |    141.40414 | 0.03163 |
|      17 |    135.28636 | 0.03165 |
|      18 |    129.47836 | 0.03139 |
|      19 |    123.95027 | 0.03209 |
|      20 |    118.6

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fecec89a128>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fecec89a0f0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

## Saving and restoring

### Save previous net

In [25]:
score_before = mean_squared_error(y, net.predict(X))
print(score_before)

115.695782722


In [26]:
with open('mynet.pkl', 'wb') as f:
    pickle.dump(net, f)

### Create a new net with same architecture

In [27]:
with open('mynet.pkl', 'rb') as f:
    new_net = pickle.load(f)

In [28]:
score_after = mean_squared_error(y, new_net.predict(X))
print(score_after)
assert np.isclose(score_before, score_after)

115.695782722
